In [1]:
# ==========================================
# 1. 라이브러리 및 환경 설정
# ==========================================
!pip install huggingface_hub -q

import torch
import torch.nn as nn
import numpy as np
import joblib
from huggingface_hub import hf_hub_download

# 실행할 때마다 같은 결과가 나오도록 시드(Seed)를 고정하기
torch.manual_seed(42)
np.random.seed(42)

In [2]:
# ==========================================
# 2. Hugging Face에서 모델 및 스케일러 다운로드
# ==========================================
repo_id = "ohsj611/batter_classifier"

print("Hugging Face에서 모델과 스케일러를 다운로드합니다.")

try:
    # 모델 가중치 다운로드
    model_path = hf_hub_download(repo_id=repo_id, filename="best_batter_classifier.pth")
    # 스케일러 다운로드
    scaler_path = hf_hub_download(repo_id=repo_id, filename="scaler.pkl")

    print("✅ 모델 다운로드 성공!")
    print(f"Model Path: {model_path}")
    print("✅ 스케일러 다운로드 성공!")
    print(f"Scaler Path: {scaler_path}")
except Exception as e:
    print(f"❌ 다운로드 실패: {e}")
    raise

Hugging Face에서 모델과 스케일러를 다운로드합니다.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ 모델 다운로드 성공!
Model Path: /root/.cache/huggingface/hub/models--ohsj611--batter_classifier/snapshots/53853c0d40c86818040b7ce2b59679c38153cb95/best_batter_classifier.pth
✅ 스케일러 다운로드 성공!
Scaler Path: /root/.cache/huggingface/hub/models--ohsj611--batter_classifier/snapshots/53853c0d40c86818040b7ce2b59679c38153cb95/scaler.pkl


In [3]:
# ==========================================
# 3. 모델 아키텍처 정의
# ==========================================

# MLP (Multi-Layer Perceptron)
class BatterTypeClassifier(nn.Module):
    def __init__(self):
        super(BatterTypeClassifier, self).__init__()
        # 입력: 5개, 출력: 64개
        self.layer1 = nn.Linear(5, 64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 3)

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.dropout(x)
        x = self.relu(self.layer2(x))
        x = self.layer3(x)
        return x

In [4]:
# ==========================================
# 4. 모델 및 스케일러 로드
# ==========================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 로드
model = BatterTypeClassifier()

checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint)

model.to(device)
model.eval()

print(f"✅ 모델 로드 성공!")

# 스케일러 로드
scaler = joblib.load(scaler_path)
print("✅ 스케일러 로드 성공!")

✅ 모델 로드 성공!
✅ 스케일러 로드 성공!


In [5]:
# ==========================================
# 5. 시스템 출력 설정
# ==========================================
def recommend_breeding_guide(stats_dict):
    print()
    print(f"{stats_dict['Name']}({stats_dict['Year']})의 타격 지표를 분석한 결과입니다.")

    # 파생 지표 계산
    try:
        pa = stats_dict['PA'] # 타석
        ab = stats_dict['AB'] # 타수

        # 타수 기준 지표
        # H_Ratio
        h_ratio = stats_dict['H'] / ab if ab > 0 else 0
        # 2B_3B_Ratio
        xb_ratio = (stats_dict['2B'] + stats_dict['3B']) / ab if ab > 0 else 0
        # HR_Ratio
        hr_ratio = stats_dict['HR'] / ab if ab > 0 else 0

        # 타석 기준 지표
        # SO_Ratio
        so_ratio = stats_dict['SO'] / pa if pa > 0 else 0
        # BB_Ratio
        bb_ratio = stats_dict['BB'] / pa if pa > 0 else 0

    except KeyError as e:
        print(f"❌ 오류 발생: 필수 데이터인 {e}가 누락되었습니다.")
        return

    # 모델 입력
    # H_Ratio, 2B_3B_Ratio, HR_Ratio, SO_Ratio, BB_Ratio
    raw_features = np.array([[h_ratio, xb_ratio, hr_ratio, so_ratio, bb_ratio]])

    # 스케일링
    scaled_features = scaler.transform(raw_features)
    inputs = torch.FloatTensor(scaled_features).to(device)

    # 모델 출력
    with torch.no_grad():
        outputs = model(inputs)
        _, predicted_idx = torch.max(outputs, 1)
        type_idx = predicted_idx.item()

    # 타격 지표 기반 타자의 타격 유형 자동 분류를 통한 이사만루 육성 가이드 시스템
    types = ['GAP', 'POWER', 'CONTACT']
    result_type = types[type_idx]

    if result_type == 'POWER':
        print(f"• 타격 유형: [ POWER Hitter ]")
        print(f"• 육성 가이드: 파워에 집중 투자하세요.")
        print(f"• 추천 포인트 분배: 파워 15 / 컨택 4 / 선구 1")

    elif result_type == 'GAP':
        print(f"• 타격 유형: [ GAP Hitter ]")
        print(f"• 육성 가이드: 컨택을 중심으로 파워와 선구의 균형을 맞추어 투자하세요.")
        print(f"• 추천 포인트 분배: 파워 5 / 컨택 10 / 선구 5")

    else: # CONTACT
        print(f"• 타격 유형: [ CONTACT Hitter ]")
        print(f"• 육성 가이드: 컨택과 선구에 집중 투자하세요.")
        print(f"• 추천 포인트 분배: 파워 1 / 컨택 10 / 선구 9")

In [6]:
# ==========================================
# 사용자 입력 설정
# ==========================================

def run_inference_system():
    """
    사용자로부터 타자 정보와 스탯을 입력받아 육성 가이드를 실행하는 메인 함수
    """
    print("\n" + "="*60)
    print("[ 타격 지표 기반 타자의 타격 유형 자동 분류를 통한 이사만루 육성 가이드 시스템 ]")
    print("="*60)

    user_stats = {}

    try:
        # 1. 이름 입력 받기
        print("\n타자의 이름을 입력한 후 Enter를 눌러주세요. (예시: 이정후)")
        name = input("• 이름 : ")
        user_stats['Name'] = name if name.strip() else "User Input Player"

        # 2. 연도 입력 받기
        print("\n입력할 타격 지표에 해당하는 연도를 입력한 후 Enter를 눌러주세요. (예시: 2023)")
        year = input("• 연도 : ")
        user_stats['Year'] = year if year.strip() else "Unknown Year"

        # 3. 타격 지표 입력 받기
        print("\n타격 지표를 입력한 후 Enter를 눌러주세요. (예시: 86)")

        input_fields = [
            ('PA', 'PA(타석)'),
            ('AB', 'AB(타수)'),
            ('H',  'H(안타)'),
            ('2B', '2B(2루타)'),
            ('3B', '3B(3루타)'),
            ('HR', 'HR(홈런)'),
            ('SO', 'SO(삼진)'),
            ('BB', 'BB(볼넷)')
        ]

        for key, label in input_fields:
            while True:
                try:
                    val = input(f"• {label:<8}: ")

                    val = int(val)

                    if val < 0:
                        print("   ❌ 양수만 입력할 수 있습니다. 다시 입력해주세요.")
                        continue

                    user_stats[key] = val
                    break
                except ValueError:
                    print("   ❌ 정수만 입력할 수 있습니다. 다시 입력해주세요.")

        recommend_breeding_guide(user_stats)

    except Exception as e:
        print(f"❌ 오류 발생: {e}")

In [7]:
# ==========================================
# [예시 1] GAP Hitter
# ==========================================

"""
[ 타격 지표 기반 타자의 타격 유형 자동 분류를 통한 이사만루 육성 가이드 시스템 ]

타자의 이름을 입력한 후 Enter를 눌러주세요. (예시: 이정후)
• 이름 : 강승호

입력할 타격 지표에 해당하는 연도를 입력한 후 Enter를 눌러주세요. (예시: 2023)
• 연도 : 2024

타격 지표를 입력한 후 Enter를 눌러주세요. (예시: 86)
• PA(타석): 566
• AB(타수): 521
• H(안타): 146
• 2B(2루타): 34
• 3B(3루타): 7
• HR(홈런): 18
• SO(삼진): 158
• BB(볼넷): 32

강승호(2024)의 타격 지표를 분석한 결과입니다.
• 타격 유형: [ GAP Hitter ]
• 육성 가이드: 컨택을 중심으로 파워와 선구의 균형을 맞추어 투자하세요.
• 추천 포인트 분배: 파워 5 / 컨택 10 / 선구 5
"""
run_inference_system()


[ 타격 지표 기반 타자의 타격 유형 자동 분류를 통한 이사만루 육성 가이드 시스템 ]

타자의 이름을 입력한 후 Enter를 눌러주세요. (예시: 이정후)
• 이름 : 강승호

입력할 타격 지표에 해당하는 연도를 입력한 후 Enter를 눌러주세요. (예시: 2023)
• 연도 : 2024

타격 지표를 입력한 후 Enter를 눌러주세요. (예시: 86)
• PA(타석)  : 566
• AB(타수)  : 521
• H(안타)   : 146
• 2B(2루타) : 34
• 3B(3루타) : 7
• HR(홈런)  : 18
• SO(삼진)  : 158
• BB(볼넷)  : 32

강승호(2024)의 타격 지표를 분석한 결과입니다.
• 타격 유형: [ GAP Hitter ]
• 육성 가이드: 컨택을 중심으로 파워와 선구의 균형을 맞추어 투자하세요.
• 추천 포인트 분배: 파워 5 / 컨택 10 / 선구 5


In [8]:
# ==========================================
# [예시 2] POWER Hitter
# ==========================================

"""
[ 타격 지표 기반 타자의 타격 유형 자동 분류를 통한 이사만루 육성 가이드 시스템 ]

타자의 이름을 입력한 후 Enter를 눌러주세요. (예시: 이정후)
• 이름 : 맷 데이비슨

입력할 타격 지표에 해당하는 연도를 입력한 후 Enter를 눌러주세요. (예시: 2023)
• 연도 : 2025

타격 지표를 입력한 후 Enter를 눌러주세요. (예시: 86)
• PA(타석): 439
• AB(타수): 386
• H(안타): 113
• 2B(2루타): 18
• 3B(3루타): 0
• HR(홈런): 36
• SO(삼진): 118
• BB(볼넷): 31

맷 데이비슨(2025)의 타격 지표를 분석한 결과입니다.
• 타격 유형: [ POWER Hitter ]
• 육성 가이드: 파워에 집중 투자하세요.
• 추천 포인트 분배: 파워 15 / 컨택 4 / 선구 1
"""
run_inference_system()


[ 타격 지표 기반 타자의 타격 유형 자동 분류를 통한 이사만루 육성 가이드 시스템 ]

타자의 이름을 입력한 후 Enter를 눌러주세요. (예시: 이정후)
• 이름 : 맷 데이비슨

입력할 타격 지표에 해당하는 연도를 입력한 후 Enter를 눌러주세요. (예시: 2023)
• 연도 : 2025

타격 지표를 입력한 후 Enter를 눌러주세요. (예시: 86)
• PA(타석)  : 439
• AB(타수)  : 386
• H(안타)   : 113
• 2B(2루타) : 18
• 3B(3루타) : 0
• HR(홈런)  : 36
• SO(삼진)  : 118
• BB(볼넷)  : 31

맷 데이비슨(2025)의 타격 지표를 분석한 결과입니다.
• 타격 유형: [ POWER Hitter ]
• 육성 가이드: 파워에 집중 투자하세요.
• 추천 포인트 분배: 파워 15 / 컨택 4 / 선구 1


In [9]:
# ==========================================
# [예시 3] CONTACT Hitter
# ==========================================

"""
[ 타격 지표 기반 타자의 타격 유형 자동 분류를 통한 이사만루 육성 가이드 시스템 ]

타자의 이름을 입력한 후 Enter를 눌러주세요. (예시: 이정후)
• 이름 : 황성빈

입력할 타격 지표에 해당하는 연도를 입력한 후 Enter를 눌러주세요. (예시: 2023)
• 연도 : 2024

타격 지표를 입력한 후 Enter를 눌러주세요. (예시: 86)
• PA(타석): 406
• AB(타수): 366
• H(안타): 117
• 2B(2루타): 15
• 3B(3루타): 8
• HR(홈런): 4
• SO(삼진): 56
• BB(볼넷): 31

황성빈(2024)의 타격 지표를 분석한 결과입니다.
• 타격 유형: [ CONTACT Hitter ]
• 육성 가이드: 컨택과 선구에 집중 투자하세요.
• 추천 포인트 분배: 파워 1 / 컨택 10 / 선구 9
"""
run_inference_system()


[ 타격 지표 기반 타자의 타격 유형 자동 분류를 통한 이사만루 육성 가이드 시스템 ]

타자의 이름을 입력한 후 Enter를 눌러주세요. (예시: 이정후)
• 이름 : 황성빈

입력할 타격 지표에 해당하는 연도를 입력한 후 Enter를 눌러주세요. (예시: 2023)
• 연도 : 2024

타격 지표를 입력한 후 Enter를 눌러주세요. (예시: 86)
• PA(타석)  : 406
• AB(타수)  : 366
• H(안타)   : 117
• 2B(2루타) : 15
• 3B(3루타) : 8
• HR(홈런)  : 4
• SO(삼진)  : 56
• BB(볼넷)  : 31

황성빈(2024)의 타격 지표를 분석한 결과입니다.
• 타격 유형: [ CONTACT Hitter ]
• 육성 가이드: 컨택과 선구에 집중 투자하세요.
• 추천 포인트 분배: 파워 1 / 컨택 10 / 선구 9
